<img src='img/anaconda-logo.png' align='left' style="padding:10px">
<br>
*Copyright Continuum 2012-2016 All Rights Reserved.*

# Accelerate GPU Programming

* Both **Numba and Accelerate** provide CUDA programming capability
* **Numba** focuses on the ability to create **custom CUDA functions** and misc CUDA operations
    * (i.e. memory allocation and transfer between CPU and GPU)
* **Accelerate** focuses on **pre-built CUDA functions** and CUDA library bindings.

## Table of Contents
* [Accelerate GPU Programming](#Accelerate-GPU-Programming)
* [Memory](#Memory)
	* [Numpy arrays in device memory](#Numpy-arrays-in-device-memory)
* [Ufuncs](#Ufuncs)
* [Writing custom CUDA functions](#Writing-custom-CUDA-functions)
	* [Thread Hierarchy](#Thread-Hierarchy)
	* [Memory Hierarchy](#Memory-Hierarchy)
	* [Exercise: Discard unneeded intermediate reductions](#Exercise:-Discard-unneeded-intermediate-reductions)


# Memory

The first major difference between CPU and GPU programming is the memory.

* Each GPU has its own memory, referred to here as the ***device memory***.
    * The device memory is on the GPU card.
    * It is separated from the ***host*** memory by the PCI-express bus.
    * The ***device (GPU)*** cannot access ***host*** memory allocated normally.  
* The CPU has access to the ***host memory***.
    * The ***host memory*** is the system primary memory (i.e. RAM).
    * The **host*** (CPU) cannot directly access ***device memory***.

***Note: CUDA driver provides special allocators to create page-locked host memory for access from device memory.  But that is not common, and is beyound the scope of this course.***

## Numpy arrays in device memory

CUDA functions in Numba and Accelerate can use Numpy arrays as arguments by implicitly transfering the array to device memory.  For performance reason, it is sometimes preferred to do explicit memory transfer.

To copy an array to the device

In [ ]:
import numpy as np
from numba import cuda

arr = np.random.random(10)   # init random array on the host
d_arr = cuda.to_device(arr)  # copy to device

print(arr, arr.shape, arr.dtype)
print(d_arr, d_arr.shape, d_arr.dtype)

**Note:**

* the device ndarray cannot be use as a numpy ndarray
* it implements a few array methods/attributes compatible with numpy

To copy a device array back to the host

In [ ]:
arr2 = d_arr.copy_to_host()   # copy to host
print(arr2)
assert np.all(arr == arr2)    # verify the data

# Ufuncs 

Ufuncs provides a high-level API for creating elementwise array functions that can target CPU and GPU.

In [ ]:
from numba import vectorize
import math

@vectorize(['float32(float32, float32)', 
            'float64(float64, float64)']) # default to cpu
def cpu_ufunc_math(x, y):
    return math.sin(x) / math.cos(y)


@vectorize(['float32(float32, float32)', 
            'float64(float64, float64)'], target='cuda')
def gpu_ufunc_math(x, y):
    return math.sin(x) / math.cos(y)

In [ ]:
arr = np.random.random(10**5).astype(np.float32)

cpu_res = cpu_ufunc_math(arr, arr)
gpu_res = gpu_ufunc_math(arr, arr)  # implicit memory transfer
assert np.allclose(cpu_res, gpu_res)

In [ ]:
print('CPU Ufunc')
%timeit cpu_ufunc_math(arr, arr)
print('\nGPU Ufunc with implicit transfer')
%timeit gpu_ufunc_math(arr, arr)

d_arr = cuda.to_device(arr)        # move data onto the device
def call_gpu_ufunc_math(d_arr):
    gpu_ufunc_math(d_arr, d_arr)   # using device memory directly; no memory transfer
    cuda.synchronize()             # ensure completion; CUDA kernel is asynchronous

print('\nGPU Ufunc directly in device memory')
%timeit call_gpu_ufunc_math(d_arr)

# Writing custom CUDA functions

The CUDA execution model defines two types of functions:

**kernel**

A kernel is callable from the host.  It is associated with a grid—a group of thread-blocks.  Since a kernel launch is asynchronous to the host, it has no return value.  The computation result must be stored into output arguments.

**device function**

A device function is callable from the device (i.e. from kernels or device functions).  It behaves like normal functions and has a return value.

## Thread Hierarchy

A kernel launches with a grid.  A grid contains thread blocks and each thread block contains threads.  This forms a three tier thread hierarchy.  The kernel code is executed once per thread.  Blocks are scheduled concurrently as resources become available.  That means not all blocks are active at the same time, so inter-block dependencies are not possible.  All threads within a block are active when the block is active.  Therefore, intra-block thread communication is possible.

A grid and a block can both be 1D, 2D, 3D. So there can be 6D to work with.  The coordinates of blocks and threads can be used to compute an identitier for the current thread.

```python
idx = cuda.threadIdx.x + cuda.blockIdx.x * cuda.blockDim.x 
idy = cuda.threadIdx.y + cuda.blockIdx.y * cuda.blockDim.y
idz = cuda.threadIdx.z + cuda.blockIdx.z * cuda.blockDim.z
```

There is a shorthand for this common task:

```python
idx = cuda.grid(1)
idx, idy = cuda.grid(2)
idx, idy, idz = cuda.grid(3)
```

![cuda grid](img/grid-of-thread-blocks.png)

See: http://docs.nvidia.com/cuda/cuda-c-programming-guide/

In [ ]:
@cuda.jit                  # kernel function
def copy_vector(data, output):
    idx = cuda.grid(1)     # get global-id for the current thread for a 1D grid
    if idx < output.size:  # if index is in-range
        output[idx] = data[idx]

In [ ]:
# prepare input and output memory
arr = np.arange(100)
out = np.zeros_like(arr)

# prepare grid/block configuration for the launch
blocksize = 128
gridsize = int(math.ceil(arr.size / blocksize))

# launch kernel
copy_vector[gridsize, blocksize](arr, out)

print(out)
# verify result
assert np.all(arr == out)

In [ ]:
@cuda.jit
def copy_vector3d(data, output):
    idx, idy, idz = cuda.grid(3)
    if idx < output.shape[0] and idy < output.shape[1] and idz < output.shape[2]:
        output[idx, idy, idz] = data[idx, idy, idz]

In [ ]:
arr = np.arange(100).reshape(4, 5, 5)
out = np.zeros_like(arr)

# block of 2 x 2 x 2 threads
blocksize = 2, 2, 2
# compute gridsize as ceil(ndata/blocksize)
gridsize = tuple(np.ceil(np.asarray(arr.shape) / np.asarray(blocksize)).astype(np.int))

copy_vector3d[gridsize, blocksize](arr, out)

print(out)
assert np.all(arr == out)

## Memory Hierarchy

In addition to the device memory (a.k.a global memory), there are other memory units.  Numba exposes the _local memory_ and _shared memory_.  The local memory is for per-thread storage with a fixed size at compile time.  The shared memory is a per-block storage with a fixed size at compile time.  It is important as a way for fast intra-block communication among threads.

![cuda memory](img/memory-hierarchy.png)

See: http://docs.nvidia.com/cuda/cuda-c-programming-guide/

To demonstrate intra-block communication, we implement a parallel reduction algorithm below.

In [ ]:
from numba import float32

# Create a device function for the operation by the reduction.
# This is separated as a device function for demonstration purpose-only
@cuda.jit(device=True)
def add(x, y):          
    return x + y

@cuda.jit               # the reduction kernel function
def block_reduce(arr, out):
    # Get thread identity
    idx = cuda.grid(1)
    tid = cuda.threadIdx.x
    blkid = cuda.blockIdx.x
    blksz = cuda.blockDim.x
    
    # Declare a shared memory.
    # Note: A shared memory is statically allocated.  
    #       The shape must be constant at compile time.
    sm = cuda.shared.array(shape=64, dtype=float32)
    # load data into SM cooperatively
    # Explained:
    #    * load data if index is in range
    #    * otherwise, initialize to 0
    sm[tid] = arr[idx] if idx < arr.size else 0

    # Core reduction logic
    step = blksz // 2      # defines the active threads; init to 1st half of the block

    while step > 0:
        # The second half is added to the first half
        if tid < step:
            # All threads within the first half of the block compute concurrently
            sm[tid] = add(sm[tid], sm[tid + step])
        # Reduce active threads by half
        step //= 2
        # Synchronize threads in the block.
        # This ensures all threads has reached this point.
        cuda.syncthreads()  

    # Save result
    if idx < out.size:
        out[idx] = sm[tid]

arr = np.ones(128, dtype=np.float32)
out = np.zeros_like(arr)
block_reduce[2, 64](arr, out)

print(out)

Note: currently the output array is showing all intermediate results of the parallel reduction.

## Exercise: Discard unneeded intermediate reductions

Change the above ``block_reduce`` kernel to store the per-block reduction result only.  The `out` array should be of length 2.  Element 0 store the sum of the first 64 elements.  Element 1 store the next 64 elements.

In [ ]:
# Your code here...

Check result with:

In [ ]:
assert out.size == 2
assert out[0] == arr[:64].sum()
assert out[1] == arr[64:].sum()

Run the below cell to see the answer

In [ ]:
answer = b'CmZyb20gbnVtYmEgaW1wb3J0IGZsb2F0MzIKCkBjdWRhLmppdChkZXZpY2U9VHJ1ZSkgICMgZGV2aWNlIGZ1bmN0aW9uCmRlZiBhZGQoeCwgeSk6CiAgICByZXR1cm4geCArIHkKCgpAY3VkYS5qaXQKZGVmIGJsb2NrX3JlZHVjZShhcnIsIG91dCk6CiAgICBpZHggPSBjdWRhLmdyaWQoMSkKICAgIHRpZCA9IGN1ZGEudGhyZWFkSWR4LngKICAgIGJsa2lkID0gY3VkYS5ibG9ja0lkeC54CiAgICBibGtzeiA9IGN1ZGEuYmxvY2tEaW0ueAogICAgCiAgICBzbSA9IGN1ZGEuc2hhcmVkLmFycmF5KHNoYXBlPTY0LCBkdHlwZT1mbG9hdDMyKQogICAgIyBsb2FkIGRhdGEgaW50byBTTSBjb29wZXJhdGl2ZWx5CiAgICBzbVt0aWRdID0gYXJyW2lkeF0gaWYgaWR4IDwgYXJyLnNpemUgZWxzZSAwCgogICAgIyByZWR1Y3Rpb24KICAgIHN0ZXAgPSBibGtzeiAvLyAyCgogICAgd2hpbGUgc3RlcCA+IDA6CiAgICAgICAgaWYgdGlkIDwgc3RlcDoKICAgICAgICAgICAgc21bdGlkXSA9IGFkZChzbVt0aWRdLCBzbVt0aWQgKyBzdGVwXSkKICAgICAgICBzdGVwIC8vPSAyCiAgICAgICAgIyBzeW5jaHJvbml6ZSB0aHJlYWRzIGluIHRoZSBibG9jawogICAgICAgIGN1ZGEuc3luY3RocmVhZHMoKSAgCgogICAgIyBTYXZlIHJlc3VsdAogICAgaWYgdGlkID09IDA6CiAgICAgICAgb3V0W2Jsa2lkXSA9IHNtW3RpZF0KCiAgICAKYXJyID0gbnAuYXJhbmdlKDEyOCwgZHR5cGU9bnAuZmxvYXQzMikKb3V0ID0gbnAuemVyb3MoMiwgZHR5cGU9bnAuZmxvYXQzMikKYmxvY2tfcmVkdWNlWzIsIDY0XShhcnIsIG91dCkKCnByaW50KG91dCkK'
import base64
print(base64.b64decode(answer).decode('ascii'))

---
<br>
*Copyright Continuum 2012-2016 All Rights Reserved.*